In [2]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

# Cake
# 1. The cake is in the kitchen
# 2. The cake is delicious
# 3. The cake is not eaten
# 4. The cake is sweet
# 5. The cake is not sweet
# 6. The cake is not delicious
# 7. The cake is eaten
# 8. The cake is in the stomach

def eat():
    return "eat_cake"

def bake():
    return "bake_cake"

stmap = { 
        Strips(eat(), { "cake": True}, {"cake": False}),
        Strips(bake(), { "cake": False}, {"cake": True})
    }

# all possible actions
feature_domain_dict = { "cake": True, "cake": False }
strips_domain = STRIPS_domain(feature_domain_dict, stmap)
cake_problem = Planning_problem(strips_domain, { "cake": False}, { "cake": True})
SearcherMPP(Forward_STRIPS(cake_problem)).search()


Solution: {'cake': False}
   --bake_cake--> {'cake': True} (cost: 1)
 2 paths have been expanded and 0 paths remain in the frontier


{'cake': False}
   --bake_cake--> {'cake': True}

In [ ]:
# magic world

# Copilot stop spamming the fucking chat
# 1. The chat is not spammed
# 2. The chat is spammed!
# 3. The chat is not spammed, the copilot is not spamming, the chat is not spammed, Copilot is the spammer

def move():
    pass

def attack():
    pass

def open():
    pass

def collectFire():
    pass

def collectEarth():
    pass

def buildFireball():
    pass

# nxn grid with player, castle, walls, and dragons
# End goal is kill the dragon and react the castle
# To kill the dragon, player needs a fireball
# To build a fireball, the player needs k earth and l fire
# To attack, the player has to be at the same position as a dragon
# 

def initWorld():
    # nxn grid
    worldSize = (5, 5)